pip install alpaca-py
pip install backtrader
pip install matplotlib
pip install backtrader-plotting
pip install numpy
pip install scikit-learn

In [ ]:
import backtrader as bt
from datetime import datetime
from alpaca.data import StockHistoricalDataClient, TimeFrame
from alpaca.data.requests import StockBarsRequest
import pandas as pd
import numpy as np
from IPython.display import Image
import sklearn


In [ ]:
import backtrader as bt

class BuyAndHold(bt.Strategy):
    def nextstart(self):
        # Buy all available cash worth of the asset
        size = int(self.broker.get_cash() / self.data.close[0])
        self.buy(size=size)

In [ ]:
data_client = StockHistoricalDataClient('api_key', 'secret_key')

def get_data(start, end):

    start_time = pd.to_datetime(start).tz_localize('America/New_York')
    end_time = pd.to_datetime(end).tz_localize('America/New_York')

    # Define the request parameters
    request_params = StockBarsRequest(
        symbol_or_symbols=['SPY'],  
        timeframe=TimeFrame.Day,     # Options: Minute, Hour, Day
        start=start_time,
        end=end_time
    )
    bars = data_client.get_stock_bars(request_params)
    df = bars.df.tz_convert('America/New_York', level=1)

    df.reset_index(inplace=True)
    df = df.rename(columns={'timestamp': 'datetime'})
    df.set_index('datetime', inplace=True)
    df.index = pd.to_datetime(df.index)

    return df

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(BuyAndHold)
df = get_data(start='2010-01-01', end='2025-02-01')
# Create a Data Feed
data = bt.feeds.PandasData(dataname=df)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set initial cash
cerebro.broker.setcash(10000.0)

# Run the strategy
cerebro.run()

# Set the path to save the image
img_path = "backtrader_plot.png"

# Generate the plot and save it
fig = cerebro.plot(style="candlestick")[0][0]
fig.savefig(img_path)

# Display the saved image
Image(filename=img_path)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
df = get_data(start='2010-01-01', end='2025-02-01')

In [ ]:
def generate_features(df):
    df['return'] = df['close'].pct_change()
    df['sma_10'] = df['close'].rolling(10).mean()
    df['sma_50'] = df['close'].rolling(50).mean()
    df['rsi'] = compute_rsi(df['close'])
    df.dropna(inplace=True)
    return df

def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [ ]:
df = generate_features(df)

In [ ]:
X = df[['return', 'sma_10', 'sma_50', 'rsi']]
y = np.where(df['close'].shift(-1) > df['close'], 1, 0)  # 1 = Buy, 0 = Sell


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
class MLTradingStrategy(bt.Strategy):
    def __init__(self):
        self.dataX = X_test  # Use test set for simulation
        self.bar = 0

    def next(self):
        if self.bar < len(self.dataX):
            features = self.dataX[self.bar].reshape(1, -1)
            prediction = model.predict(features)  # Predict next movement
            
            if prediction == 1 and not self.position:
                self.buy()
            elif prediction == 0 and self.position:
                self.sell()
                
            self.bar += 1


In [ ]:
class SmaStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=15)

    def next(self):
        if self.data.close[0] > self.sma[0]:
            self.buy()
        elif self.data.close[0] < self.sma[0]:
            self.sell()


In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SmaStrategy)

# Add data
data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)

# Set initial cash
cerebro.broker.setcash(1000.0)

# Add performance analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analysis')

# Run backtest
results = cerebro.run()
strategy = results[0]

# Print performance metrics
sharpe = strategy.analyzers.sharpe.get_analysis()
returns = strategy.analyzers.returns.get_analysis()
drawdown = strategy.analyzers.drawdown.get_analysis()
trades = strategy.analyzers.trade_analysis.get_analysis()

print("\n=== Backtest Results ===")
print(f"Sharpe Ratio: {sharpe.get('sharperatio', 'N/A')}")
print(f"Annual Returns: {returns}")
print(f"Max Drawdown: {drawdown.get('max', 'N/A')}%")
print(f"Total Trades: {trades.get('total', 'N/A')}")
print(f"Winning Trades: {trades.get('won', {}).get('total', 0)}")
print(f"Losing Trades: {trades.get('lost', {}).get('total', 0)}")

# Plot results
img_path = "backtrader_plot.png"
fig = cerebro.plot(style="candlestick")[0][0]
fig.set_size_inches(12, 8)  # Widen the figure to prevent clipping
fig.savefig(img_path, dpi=300, bbox_inches='tight')

In [ ]:
Image(filename=img_path)